In [1]:
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import numpy as np
from keras import backend as K
import keras
import cv2
from Utils import *
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import *
from keras.applications.mobilenet_v2 import MobileNetV2
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.models import model_from_json
import matplotlib.pyplot as plt
import os
from PIL import Image
import pytesseract

Using TensorFlow backend.


In [2]:
directory = "/home/nitin/Desktop/banglore/fresh_solution/package_images/"

for folder in tqdm(os.listdir(directory)):
    for img in tqdm(os.listdir(directory+folder)):
        if not str(folder).startswith("."):

            img_name = directory+folder+"/"+img

            img = cv2.imread(img_name) 
            stretch_near = cv2.resize(img, (1000, 1000),interpolation = cv2.INTER_NEAREST) 
            im = cv2.bilateralFilter(np.array( img,dtype=np.uint8),9,70,70)
            imb = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            im = cv2.adaptiveThreshold(imb, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 35 ,3)

            filename = img_name.split(".")[0] + "_new." + img_name.split(".")[1]
            cv2.imwrite(filename, im) 

100%|██████████| 3/3 [00:00<00:00, 61.07it/s]

100%|██████████| 3/3 [00:00<00:00, 74.01it/s]

100%|██████████| 2/2 [00:00<00:00, 54.96it/s]

100%|██████████| 5/5 [00:00<00:00, 25.00it/s]


In [3]:
#Variable Definition
img_w = 512
img_h = 512
channels = 3
classes = 1
info = 5
grid_w = 16
grid_h = 16

In [4]:
def load_model(strr):        
    json_file = open(strr, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    return loaded_model


In [5]:
model = load_model('model1/text_detect_model.json')
model.load_weights('model1/text_detect.h5')

In [6]:
def predict_func(model , inp , iou , name, l, k):

    ans = model.predict(inp)
    
    #np.save('Results/ans.npy',ans)
    boxes = decode(ans[0] , img_w , img_h , iou)
    
    img = ((inp + 1)/2)
    img = img[0]
    #plt.imshow(img)
    #plt.show()
    j=1
    
    for i in boxes:
    
        i = [int(x) for x in i]

        img = cv2.rectangle(img , (i[0] ,i[1]) , (i[2] , i[3]) , color = (0,255,0) , thickness = 2)
        crop_img = img[i[1]:i[3], i[0]:i[2]]

        
        cv2.imwrite(os.path.join(k , l.split(".")[0] + "_"+ str(j)+'.jpg') , crop_img*255.0)
        #plt.imshow(crop_img)
        #plt.show()
        j = j+1
        

         
     


In [8]:
directory = "/home/nitin/Desktop/banglore/fresh_solution/package_images/"

for folder in tqdm(os.listdir(directory)):
    os.mkdir("/home/nitin/Desktop/banglore/fresh_solution/Results/"+folder)

    for img in tqdm(os.listdir(directory+folder)):
        if not str(folder).startswith(".") and "new" in str(img):

            k = "/home/nitin/Desktop/banglore/fresh_solution/Results/"+folder+"/"+img.split(".")[0]
            l = str(img)
            os.mkdir(k)


            img_name = directory+folder+"/"+img

            print(img_name)

            
            img = cv2.imread(img_name) 
            img = cv2.resize(img,(512,512))
            img = (img - 127.5)/127.5
            
            predict_func(model , np.expand_dims(img,axis= 0) , 0.5 , 'sample', l, k)
            
        
            

  0%|          | 0/6 [00:00<?, ?it/s]

/home/nitin/Desktop/banglore/fresh_solution/package_images/3/02_new.jpg



 17%|█▋        | 1/6 [00:01<00:07,  1.53s/it]

/home/nitin/Desktop/banglore/fresh_solution/package_images/3/01_new.jpg



  0%|          | 0/6 [00:00<?, ?it/s]

...


  0%|          | 0/6 [00:00<?, ?it/s]

/home/nitin/Desktop/banglore/fresh_solution/package_images/1/2_new.jpeg



 83%|████████▎ | 5/6 [00:00<00:00,  6.16it/s]

/home/nitin/Desktop/banglore/fresh_solution/package_images/1/01_new.jpeg
/home/nitin/Desktop/banglore/fresh_solution/package_images/1/03_new.jpeg



100%|██████████| 5/5 [00:03<00:00,  1.27it/s]


In [9]:
Text = []
File = []

directory = "/home/nitin/Desktop/banglore/fresh_solution/Results/"

for folder in tqdm(os.listdir(directory)):
    for folder2 in tqdm(os.listdir(directory+folder)):
        for img in tqdm(os.listdir(directory+folder+"/"+folder2)):
            if not str(folder).startswith("."):
                #print(directory+folder+"/"+folder2+"/"+img)
                im = Image.open(directory+folder+"/"+folder2+"/"+img)
                newsize = (1000, 1000) 
                im1 = im.resize(newsize)
                text = pytesseract.image_to_string(im1,lang="eng")
                Text.append(text)
                File.append(folder+"/"+folder2+"/"+img)
                
    
    
            


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [00:00<00:02,  2.11it/s]

 33%|███▎      | 2/6 [00:00<00:01,  2.32it/s]

 50%|█████     | 3/6 [00:01<00:01,  2.62it/s]

 67%|██████▋   | 4/6 [00:01<00:00,  2.78it/s]

 83%|████████▎ | 5/6 [00:01<00:00,  2.85it/s]

100%|██████████| 6/6 [00:02<00:00,  2.98it/s]

 33%|███▎      | 1/3 [00:02<00:04,  2.02s/it]

  0%|          | 0/5 [00:00<?, ?it/s]

 20%|██        | 1/5 [00:00<00:01,  2.75it/s]

 40%|████      | 2/5 [00:00<00:01,  2.91it/s]

 60%|██████    | 3/5 [00:01<00:00,  2.51it/s]

 80%|████████  | 4/5 [00:01<00:00,  2.82it/s]

100%|██████████| 5/5 [00:01<00:00,  2.62it/s]

 67%|██████▋   | 2/3 [00:03<00:01,  1.99s/it]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  3.59it/s]

 50%|█████     | 2/4 [00:00<00:00,  3.54it/s]

 75%|███████▌  | 3/4 [00:00<00:00,  3.21it/s]

100%|██████████| 4/4 [00:01<00:00,  2.98it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%| 

In [10]:
text = pd.DataFrame(Text)
file = pd.DataFrame(File)
df = pd.concat([file,text],axis =1)
df.to_csv("new1.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>